Alright folks, this will be a tutorial that walks you through how to build your own Full Text Search engine from scratch.

By completing this tutorial, you will understand Tokenizers, Analyzers, Indexing, and Search. 

## Analyze

Installing all the prerequisites

In [ ]:
! pip install pystemmer

Importing some of the libraries we will be using.

**Stemmer:** which is basically just removing the suffix from a word and reduce it to its root word. For example: “Flying” is a word and its suffix is “ing”, if we remove “ing” from “Flying” then we will get base word or root word which is “Fly”.

**re:** Python's default regex library

**string:** Python's default string manipulation library

In [ ]:
import Stemmer
import re
import string

In [ ]:
# simply breaking a string up by whitespace into an array of strings.
def tokenize(text):
    return text.split()

In [ ]:
# converting every string into lowercarse
def lowercase_filter(tokens):
    return [token.lower() for token in tokens]

In [ ]:
# applying the stemmer library to get every word to its' root
def stem_filter(tokens):
    STEMMER = Stemmer.Stemmer('english')
    return STEMMER.stemWords(tokens)

In [ ]:
# remove all punctuation
def punctuation_filter(tokens):
    PUNCTUATION = re.compile('[%s]' % re.escape(string.punctuation))
    return [PUNCTUATION.sub('', token) for token in tokens]

These are the top 25 most common words in English according to wikipedia:
https://en.wikipedia.org/wiki/Most_common_words_in_English

In [ ]:
# remove all stop words
def stopword_filter(tokens):
    STOPWORDS = set(['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'have',
                     'I', 'it', 'for', 'not', 'on', 'with', 'he', 'as', 'you',
                     'do', 'at', 'this', 'but', 'his', 'by', 'from', 'wikipedia'])
    return [token for token in tokens if token not in STOPWORDS]

Now an "analyze" function which puts all of the functions above into one single function:

In [ ]:
def analyze(text):
    tokens = tokenize(text)
    tokens = lowercase_filter(tokens)
    tokens = punctuation_filter(tokens)
    tokens = stopword_filter(tokens)
    tokens = stem_filter(tokens)

    return [token for token in tokens if token]

So let's test it using a common sentence.

In [ ]:
analyze("The quick brown fox jumps over the lazy dog")

## Index

In [ ]:
import json

# importing the movies collection as a dictionary
filename = 'data/movies.json'
with open(filename, 'r') as f:
    documents = json.load(f)

def index():
    index = {}
    # for each movie, run the analyzer function above on title and add it to a set with the movies' ID
    for document in documents:
        for token in analyze(document['title']):
            index[token] = set()
            index[token].add(document['_id']['$oid'])
            
    return index

## Search

In [ ]:
def search(query):
    # tokenize the query     
    analyzed_query = analyze(query)
    # grab movie tokens from the index that match the tokens from the query    
    results = [index().get(token, set()) for token in analyzed_query]
    
    resulting_documents = []
    
    # return all movies where the tokenized query matches the tokenized title
    for result in results:
        result_str = ', '.join(result)
        for document in documents:
            if document['_id']['$oid'] == result_str:
                resulting_documents.append(document)
    return resulting_documents
    
search("forrest gump")

## Relevance

In [ ]:
# index.py
import math

def document_frequency(self, token):
    return len(self.index.get(token, set()))

def inverse_document_frequency(self, token):
    # Manning, Hinrich and Schütze use log10, so we do too, even though it
    # doesn't really matter which log we use anyway
    # https://nlp.stanford.edu/IR-book/html/htmledition/inverse-document-frequency-1.html
    return math.log10(len(self.documents) / self.document_frequency(token))

def rank(self, analyzed_query, documents):
    results = []
    if not documents:
        return results
    for document in documents:
        score = 0.0
        for token in analyzed_query:
            tf = document.term_frequency(token)
            idf = self.inverse_document_frequency(token)
            score += tf * idf
        results.append((document, score))
    return sorted(results, key=lambda doc: doc[1], reverse=True)
    